In [75]:
import torch
import torchani
import ase
import ase.data
import ase.optimize
import ase.vibrations
import json

In [71]:
import scipy
import scipy.special
#import scipy.special.binom
scipy.special.binom(8, 4) * 286

20020.0

In [ ]:
ase.

In [61]:
with open('/home/alessandro/Downloads/ethane.cjson') as f:
    cjson = json.load(f)

In [140]:
def get_ase_atoms(cjson_atoms):
    elements = cjson_atoms.get('elements', {})
    number = elements.get('number')
    if number is None:
        symbol = elements.get('symbol')
        if symbol is None:
            raise Exception('Atomic numbers are missing.')
        number = list(map(lambda sym : ase.data.atomic_numbers[sym], symbol))
    n_atoms = len(number)
    positions = cjson_atoms.get('coords', {}).get('3d', [])
    if len(positions) != 3 * n_atoms:
        raise Exception('Atomic positions are missing.')
    atoms = []
    for i in range(n_atoms):
        atoms.append(ase.Atom(number[i], positions[i * 3 : i * 3 + 3]))
    return atoms

def get_cjson_atoms(ase_molecule):
    atoms = {
        'coords': {
            '3d': list(ase_molecule.get_positions().flatten())
        },
        'elements': {
            'number': list(ase_molecule.get_atomic_numbers())
        }
    }
    return atoms

def get_cjson_vibrations(ase_vibrations):
    n_frequencies = len(ase_vibrations.get_frequencies())
    vibrations = {
        'frequencies': [freq.real for freq in ase_vibrations.get_frequencies()],
        'intensities': [1 for i in range(n_frequencies)],
        'modes': list(range(n_frequencies)),
        'eigenVectors': [list(vib.get_mode(i).flatten()) for i in range(n_frequencies)]
    }
    return vibrations

In [143]:
atoms = get_ase_atoms(cjson.get('atoms', {}))
calculator = torchani.models.ANI1ccx().ase()
calculator.device = torch.device('cpu')
molecule = ase.Atoms(atoms, calculator=calculator)

In [93]:
molecule.get_potential_energy()

-2169.4247092252085

In [96]:
ase.optimize.BFGS(molecule).run(fmax=0.0005)

      Step     Time          Energy         fmax
BFGS:    0 10:10:59    -2169.424709        0.0001


True

In [89]:
opt.run(fmax=0.001)

      Step     Time          Energy         fmax
BFGS:    0 10:07:04    -2169.382531        1.0041
BFGS:    1 10:07:04    -2169.416977        0.2255
BFGS:    2 10:07:04    -2169.421884        0.2190
BFGS:    3 10:07:04    -2169.424448        0.0241
BFGS:    4 10:07:04    -2169.424501        0.0214
BFGS:    5 10:07:04    -2169.424638        0.0315
BFGS:    6 10:07:04    -2169.424693        0.0256
BFGS:    7 10:07:04    -2169.424708        0.0068
BFGS:    8 10:07:04    -2169.424709        0.0007


True

In [132]:
list(range(5))

[0, 1, 2, 3, 4]

In [108]:
get_cjson_atoms(molecule)

{'coords': {'3d': [-1.1112093830802183e-15,
   1.0989442434119487e-15,
   0.7616506004930851,
   4.361237659819565e-16,
   -2.9989762853506413e-16,
   -0.7616506004930849,
   -0.7187301816637649,
   0.7187301816637648,
   -1.1583496619933593,
   -0.26307350495554244,
   -0.9818036866193076,
   -1.158349661993359,
   0.9818036866193076,
   0.2630735049555423,
   -1.1583496619933589,
   0.2630735049555421,
   0.9818036866193071,
   1.1583496619933586,
   -0.981803686619307,
   -0.26307350495554216,
   1.1583496619933586,
   0.718730181663765,
   -0.7187301816637651,
   1.1583496619933598]},
 'elements': {'number': [6, 6, 1, 1, 1, 1, 1, 1]}}

In [44]:
atoms.get_potential_energy(), atoms.get_positions()

(-3080.760161188891, array([[ 0.        ,  0.        , -0.06804198],
        [ 0.        ,  0.        ,  1.06804198]]))

In [46]:
vib = ase.vibrations.Vibrations(atoms)

In [47]:
vib.run()

Writing vib.eq.pckl
Writing vib.0x-.pckl
Writing vib.0x+.pckl
Writing vib.0y-.pckl
Writing vib.0y+.pckl
Writing vib.0z-.pckl
Writing vib.0z+.pckl
Writing vib.1x-.pckl
Writing vib.1x+.pckl
Writing vib.1y-.pckl
Writing vib.1y+.pckl
Writing vib.1z-.pckl
Writing vib.1z+.pckl


In [49]:
vib.summary()

---------------------
  #    meV     cm^-1
---------------------
  0    0.0i      0.0i
  1    0.0i      0.0i
  2    0.0       0.0 
  3    1.8      14.3 
  4    1.8      14.3 
  5  288.9    2330.2 
---------------------
Zero-point energy: 0.146 eV


In [59]:
vib.get_frequencies()

array([0.00000000e+00+2.27335628e-05j, 0.00000000e+00+7.74048954e-06j,
       8.58529494e-08+0.00000000e+00j, 1.42630597e+01+0.00000000e+00j,
       1.42630597e+01+0.00000000e+00j, 2.33017186e+03+0.00000000e+00j])

In [126]:
n_frequencies = len(vib.get_frequencies())
modes = [list(vib.get_mode(i).flatten()) for i in range(n_frequencies)]

In [141]:
get_cjson_vibrations(vib)

{'frequencies': [0.0,
  0.0,
  8.58529493829796e-08,
  14.263059721609435,
  14.263059721613807,
  2330.1718597699896],
 'intensities': [1, 1, 1, 1, 1, 1],
 'modes': [0, 1, 2, 3, 4, 5],
 'eigenVectors': [[0.0,
   0.05136625154948422,
   -0.18183245960989786,
   0.0,
   0.05136625154917987,
   -0.18183245960989777],
  [-0.0,
   -0.1818324596099692,
   -0.0513662515493104,
   0.0,
   -0.18183245960984423,
   -0.051366251549310384],
  [-0.18894849871330582, 0.0, 0.0, -0.18894849871330582, -0.0, 0.0],
  [0.0,
   -0.2180721881336718,
   -1.2813696083411102e-13,
   0.0,
   0.16371429787338868,
   -1.281237409712108e-13],
  [-0.2180721881337722, 0.0, 0.0, 0.16371429787328823, 0.0, 0.0],
  [0.0, 0.0, 0.21807218813377222, 0.0, 0.0, -0.16371429787328828]]}

In [145]:
num = list(molecule.get_atomic_numbers())

In [150]:
int(num[0])

6